In [33]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [34]:
df = pd.read_csv(r"D:\shalini\HS_DataClean_Nov.csv")

In [36]:
# Step 1: Read the config.json file
with open("D:\\shalini\\metrics_calculations.json", 'r') as file:
    config_data = json.load(file)

# Step 2: Extract config1 and config2 from the file
display_name_mapping = config_data['display_name_mapping']
level_1_metrics = config_data['level_1_metrics']
level_2_metrics = config_data['level_2_metrics']
level_3_metrics = config_data['level_3_metrics']
level_4_metrics = config_data['level_4_metrics']
level_5_metrics = config_data['level_5_metrics']
feature_groups = config_data['feature_groups']
config1 = config_data['config1']
config2 = config_data['config2']
config3 = config_data['config3']

# Step 3: Verify the contents
print(display_name_mapping)
print(level_1_metrics)
print(level_2_metrics)
print(level_3_metrics)
print(level_4_metrics)
print(level_5_metrics)
print(feature_groups)
print(config1)
print(config2)
print(config3)

{'DR_Visits': 'Visits', 'Unique Visitors': 'Visitors', 'Page Views': 'Page Views', 'Average Time Spent on Page': 'Average Time Spent (mins)', 'DR_New Visitors': 'New Visitors', 'DR_Server Errors': 'Server Errors', 'DR_Overall Chat': 'Visits to Chat', 'DR_continuing chat from new chat page': 'Continuing chat from new chat page', 'DR_Message Us': 'Visits to Chat (Bot)', 'DR_Agent Chat': 'Visits to Agent Chat (Skip bot)', 'DR_Self Service Initiated': 'Visits to Self Service Features', 'DR_CoxApp | Visits with Chat and No Self Service Initiated': 'Visits with Chat but No Self Service', 'DR_CoxApp | Visits with No Self Service and No Chat': 'Visits with Neither Self Service nor Chat', 'DR_CoxApp | Visits not logged in': 'Not Logged-in Visits', 'DR_Login Page Visits': 'Login Page Visits', 'DR_Successful logins': 'Login Success Rate', 'DR_Auto Logins': 'Share of Auto Login (Remember/Biometrics)', 'DR_Clicks to MFA Verification': 'with TSV', 'DR_Clicks on Forgot User ID': 'User ID Recovery', '

In [37]:
df['display_names'] = df['Metrics'].map(display_name_mapping)

In [38]:
df['level_1'] = df['display_names'].apply(lambda x: x if x in level_1_metrics else None)
df['level_2'] = df['display_names'].apply(lambda x: x if x in level_2_metrics else None)
df['level_3'] = df['display_names'].apply(lambda x: x if x in level_3_metrics else None)
df['level_4'] = df['display_names'].apply(lambda x: x if x in level_4_metrics else None)
df['level_5'] = df['display_names'].apply(lambda x: x if x in level_5_metrics else None)

In [39]:
feature_column = 'Unknown'
df['Feature'] = feature_column
for group, metrics in feature_groups.items():
   df['Feature'] = np.where(df['Metrics'].isin(metrics), group, df['Feature'])

In [40]:
def move_row(df, from_idx, to_idx):
    row = df.iloc[from_idx]
    df = df.drop(from_idx).reset_index(drop=True)
    df = pd.concat([df.iloc[:to_idx], row.to_frame().T, df.iloc[to_idx:]]).reset_index(drop=True)
    return df

df = move_row(df, 50, 46)
df = move_row(df, 51, 47)
df = move_row(df, 52, 48)
df = move_row(df, 53, 49)
df = move_row(df, 164, 158)
df = move_row(df, 165, 159)
df = move_row(df, 180, 174)
df = move_row(df, 181, 175)
df = move_row(df, 218, 206)
df = move_row(df, 219, 207)
df = move_row(df, 220, 208)
df = move_row(df, 221, 209)
df = move_row(df, 280, 268)
df = move_row(df, 281, 269)
df = move_row(df, 282, 270)
df = move_row(df, 283, 271)


In [41]:
df['unique_identifier'] = df['Feature'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))

In [42]:
def generate_hierarchy_id_grouped_by_os(df):
    df['Hierarchy_ID'] = None

   
    df['Feature_Order'] = df.groupby('Operating System Type')['unique_identifier'].transform(lambda x: pd.Series(range(len(x)), index=x.index))

    grouped = df.groupby(['Operating System Type', 'unique_identifier'], group_keys=False)
    results = []

   
    for (os_name, unique_id), group in grouped:
        current_ids = {'level_1': 0, 'level_2': 0, 'level_3': 0, 'level_4': 0, 'level_5': 0}  

        group = group.sort_values('Feature_Order')  

        for index, row in group.iterrows():
            if pd.isna(row['display_names']):
                continue  

            if not pd.isna(row['level_1']):  
                current_ids['level_1'] += 1
                current_ids['level_2'] = 0  
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}"

            elif not pd.isna(row['level_2']):  
                current_ids['level_2'] += 1
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}"

            elif not pd.isna(row['level_3']):  
                current_ids['level_3'] += 1
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}"

            elif not pd.isna(row['level_4']):  
                current_ids['level_4'] += 1
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}"

            elif not pd.isna(row['level_5']):  
                current_ids['level_5'] += 1
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}.{current_ids['level_5']}"

        results.append(group)

   
    df = pd.concat(results, ignore_index=True)

    
    #df = df[df['Display_Names'].notna()]

    
    cols = [col for col in df.columns if col != 'Hierarchy_ID' and col != 'Feature_Order'] + ['Hierarchy_ID']
    return df[cols]


df = generate_hierarchy_id_grouped_by_os(df)


In [43]:
df = generate_hierarchy_id_grouped_by_os(df)

def assign_parent_id(row):
    if not pd.isna(row['level_5']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_4']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_3']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_2']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_1']): 
        return None
    else:
        return None


df['Parent_ID'] = df.apply(assign_parent_id, axis=1)


In [44]:
df['metric_id'] = range(1, len(df) + 1)

In [45]:
df['metric_sequence_num'] = df.groupby('Operating System Type').cumcount() + 1

In [46]:
def calculate_level_no(row):
   
    if pd.isna(row['Hierarchy_ID']) or pd.isna(row['Parent_ID']):
        return 1
    
   
    return row['Hierarchy_ID'].count('.') + 1


df['level_no'] = df.apply(calculate_level_no, axis=1)


In [47]:
feature_names = df['Feature'].unique()
feature_mapping = {feature_name: idx + 1 for idx, feature_name in enumerate(feature_names)}

df['feature_id'] = df['Feature'].map(feature_mapping)


In [48]:
df['create_dt'] = pd.to_datetime('today').normalize().date()

In [49]:
df = df.rename(columns={'Parent_ID': 'parent_id_old'})

In [50]:
def calculate_parent_id(row, df):
    
    if pd.isna(row['Hierarchy_ID']):
        return None
   
    hierarchy_parts = row['Hierarchy_ID'].rsplit('.', 1)
    if len(hierarchy_parts) > 1:
       
        parent_hierarchy = hierarchy_parts[0]
        
        parent_row = df[df['Hierarchy_ID'] == parent_hierarchy]
        if not parent_row.empty:
            return parent_row['metric_id'].values[0]

    return None

df['parent_id'] = df.apply(calculate_parent_id, axis=1, df=df)


df.loc[df['Hierarchy_ID'].isna(), 'parent_id'] = None


In [51]:
df = df.rename(columns={
    'Metrics': 'metrics',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Hierarchy_ID': 'hierarchy_id',
})

In [63]:
first_table_columns = ['metric_id', 'metrics','operating_system_type','display_names', 'feature_name', 'unique_identifier', 'level_1', 'level_2', 'level_3', 'level_4','level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'level_no', 'feature_id', 'parent_id','create_dt' ]
first_table_df = df[first_table_columns]

In [64]:
#first_table_df.to_csv('hs_newfeature.csv', index = False)

In [59]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]
print(date_columns)
print(len(date_columns))

['29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-1

In [66]:
second_table_columns1 = ['metric_id', 'create_dt', 'metrics', 'operating_system_type']
second_table_columns = second_table_columns1 + date_columns
second_table_df = df[second_table_columns]

In [68]:
filtered_date_columns = second_table_df.columns[4:].to_list()
print(filtered_date_columns)

['29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-1

In [69]:
id_vars = ['metric_id', 'create_dt']

In [70]:
second_table_df_melted = second_table_df.melt(id_vars = id_vars, value_vars = filtered_date_columns, var_name = 'Date', value_name = 'Value')

In [74]:
second_table_df_melted.shape

(41670, 4)

In [75]:
#second_table_df_melted.to_csv('hs_all_feature_tab2.csv', index = False)

In [76]:
percentage_columns = []

# Iterate through each row and calculate the percentage for each DATE column
for idx, row in df.iterrows():
    # Iterate through all the DATE columns
    for date_col in date_columns:
        # Get the numerator (DATE value)
        numerator = row[date_col]
        
        # Get the parent hierarchy ID
        parent_hierarchyid = row['parent_id_old']
        
        # Check if the parent is empty
        if pd.isna(parent_hierarchyid) or parent_hierarchyid == "":
            # If the PARENTID is empty, keep the original DATE value
            percentage = numerator  # Assuming 100% when no parent is found
        else:
            # Find the row where HIERARCHYID == parent_hierarchyid to get the parent's DATE value
            parent_row = df[df['hierarchy_id'] == parent_hierarchyid]
            
            # If the parent exists, get the parent's DATE value for the current DATE column
            if not parent_row.empty:
                denominator = parent_row[date_col].values[0]
            else:
                denominator = 0  # In case no parent is found, avoid division by zero
            
            # Calculate the percentage
            if denominator != 0:
                percentage = (numerator / denominator) * 100
            else:
                percentage = 0  # To handle division by zero if no parent found
        
        # Round to 3 decimal places
        percentage = round(percentage, 3)
        
        # Add the percentage to the DataFrame in a new column
        percentage_column_name = f"{date_col} PERCENTAGE"
        if percentage_column_name not in df.columns:
            df[percentage_column_name] = None  # Initialize the column
        
        # Set the percentage for the current row and current date column
        df.at[idx, percentage_column_name] = percentage


In [89]:
#df.to_csv('hs_all_feature_perc.csv', index = False)

In [90]:
smart_help_df = df[df['feature_name'].isin(['SMART HELP','OVER ALL', 'LOG IN'])]

In [101]:
all_feature_df = df[['display_names', 'operating_system_type','29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-11-2024', '09-11-2024', '10-11-2024', '11-11-2024', '12-11-2024', '13-11-2024', '14-11-2024', '15-11-2024', '16-11-2024', '17-11-2024', '18-11-2024', '19-11-2024', '20-11-2024', '21-11-2024', '22-11-2024', '23-11-2024', '24-11-2024', '25-11-2024', '26-11-2024']]

In [91]:
sample_percent_data_df = smart_help_df[['display_names', 'operating_system_type','29-08-2024 PERCENTAGE', '30-08-2024 PERCENTAGE', '31-08-2024 PERCENTAGE', '01-09-2024 PERCENTAGE', '02-09-2024 PERCENTAGE', '03-09-2024 PERCENTAGE', '04-09-2024 PERCENTAGE', '05-09-2024 PERCENTAGE', '06-09-2024 PERCENTAGE', '07-09-2024 PERCENTAGE', '08-09-2024 PERCENTAGE', '09-09-2024 PERCENTAGE', '10-09-2024 PERCENTAGE', '11-09-2024 PERCENTAGE', '12-09-2024 PERCENTAGE', '13-09-2024 PERCENTAGE', '14-09-2024 PERCENTAGE', '15-09-2024 PERCENTAGE', '16-09-2024 PERCENTAGE', '17-09-2024 PERCENTAGE', '18-09-2024 PERCENTAGE', '19-09-2024 PERCENTAGE', '20-09-2024 PERCENTAGE', '21-09-2024 PERCENTAGE', '22-09-2024 PERCENTAGE', '23-09-2024 PERCENTAGE', '24-09-2024 PERCENTAGE', '25-09-2024 PERCENTAGE', '26-09-2024 PERCENTAGE', '27-09-2024 PERCENTAGE', '28-09-2024 PERCENTAGE', '29-09-2024 PERCENTAGE', '30-09-2024 PERCENTAGE', '01-10-2024 PERCENTAGE', '02-10-2024 PERCENTAGE', '03-10-2024 PERCENTAGE', '04-10-2024 PERCENTAGE', '05-10-2024 PERCENTAGE', '06-10-2024 PERCENTAGE', '07-10-2024 PERCENTAGE', '08-10-2024 PERCENTAGE', '09-10-2024 PERCENTAGE', '10-10-2024 PERCENTAGE', '11-10-2024 PERCENTAGE', '12-10-2024 PERCENTAGE', '13-10-2024 PERCENTAGE', '14-10-2024 PERCENTAGE', '15-10-2024 PERCENTAGE', '16-10-2024 PERCENTAGE', '17-10-2024 PERCENTAGE', '18-10-2024 PERCENTAGE', '19-10-2024 PERCENTAGE', '20-10-2024 PERCENTAGE', '21-10-2024 PERCENTAGE', '22-10-2024 PERCENTAGE', '23-10-2024 PERCENTAGE', '24-10-2024 PERCENTAGE', '25-10-2024 PERCENTAGE', '26-10-2024 PERCENTAGE', '27-10-2024 PERCENTAGE', '28-10-2024 PERCENTAGE', '29-10-2024 PERCENTAGE', '30-10-2024 PERCENTAGE', '31-10-2024 PERCENTAGE', '01-11-2024 PERCENTAGE', '02-11-2024 PERCENTAGE', '03-11-2024 PERCENTAGE', '04-11-2024 PERCENTAGE', '05-11-2024 PERCENTAGE', '06-11-2024 PERCENTAGE', '07-11-2024 PERCENTAGE', '08-11-2024 PERCENTAGE', '09-11-2024 PERCENTAGE', '10-11-2024 PERCENTAGE', '11-11-2024 PERCENTAGE', '12-11-2024 PERCENTAGE', '13-11-2024 PERCENTAGE', '14-11-2024 PERCENTAGE', '15-11-2024 PERCENTAGE', '16-11-2024 PERCENTAGE', '17-11-2024 PERCENTAGE', '18-11-2024 PERCENTAGE', '19-11-2024 PERCENTAGE', '20-11-2024 PERCENTAGE', '21-11-2024 PERCENTAGE', '22-11-2024 PERCENTAGE', '23-11-2024 PERCENTAGE', '24-11-2024 PERCENTAGE', '25-11-2024 PERCENTAGE', '26-11-2024 PERCENTAGE']]

In [92]:
sample_actual_data_df = smart_help_df[['display_names', 'operating_system_type','29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-11-2024', '09-11-2024', '10-11-2024', '11-11-2024', '12-11-2024', '13-11-2024', '14-11-2024', '15-11-2024', '16-11-2024', '17-11-2024', '18-11-2024', '19-11-2024', '20-11-2024', '21-11-2024', '22-11-2024', '23-11-2024', '24-11-2024', '25-11-2024', '26-11-2024']]

In [93]:
# Function to calculate both metrics (this part remains unchanged)
def calculate_both_metrics_1(df, new_row, denominator_ios, denominator_android, numerator_ios, numerator_android):
    date_columns = [col for col in df.columns if col not in ['display_names', 'operating_system_type']]
    for date in date_columns:
        denominator_ios_value = denominator_ios[date].values[0]
        denominator_android_value = denominator_android[date].values[0]
        numerator_ios_value = numerator_ios[date].values[0]
        numerator_android_value = numerator_android[date].values[0]
        
        result = (numerator_ios_value * denominator_ios_value + numerator_android_value * denominator_android_value) / (denominator_ios_value + denominator_android_value)
        new_row[date] = result
    
    new_row_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_row_df], ignore_index=True)
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    
    return new_row_df

In [95]:
# Initialize an empty list to store the results DataFrames
combined_dfs1 = []

# Loop over the config and dynamically fetch numerators, denominators, and new rows
for item in config1:
    numerator_ios = sample_percent_data_df[(sample_percent_data_df['display_names'] == item['numerator_ios']) & (sample_percent_data_df['operating_system_type'] == 'Apple iOS')]
    numerator_android = sample_percent_data_df[(sample_percent_data_df['display_names'] == item['numerator_android']) & (sample_percent_data_df['operating_system_type'] == 'Google Android')]
    
    denominator_ios = sample_percent_data_df[(sample_percent_data_df['display_names'] == item['denominator_ios']) & (sample_percent_data_df['operating_system_type'] == 'Apple iOS')]
    denominator_android = sample_percent_data_df[(sample_percent_data_df['display_names'] == item['denominator_android']) & (sample_percent_data_df['operating_system_type'] == 'Google Android')]

    # Pass the current configuration to the function
    result_df = calculate_both_metrics_1(
        sample_percent_data_df, 
        item['new_row'], 
        denominator_ios, 
        denominator_android, 
        numerator_ios, 
        numerator_android
    )
    
    # Append the result to the list of DataFrames
    combined_dfs1.append(result_df)

# Concatenate all DataFrames into one
combined_df1 = pd.concat(combined_dfs1, ignore_index=True)

# Show the first 20 rows of the combined DataFrame
combined_df1.head(20)

,display_names,operating_system_type,29-08-2024 PERCENTAGE,30-08-2024 PERCENTAGE,31-08-2024 PERCENTAGE,01-09-2024 PERCENTAGE,02-09-2024 PERCENTAGE,03-09-2024 PERCENTAGE,04-09-2024 PERCENTAGE,05-09-2024 PERCENTAGE,06-09-2024 PERCENTAGE,07-09-2024 PERCENTAGE,08-09-2024 PERCENTAGE,09-09-2024 PERCENTAGE,10-09-2024 PERCENTAGE,11-09-2024 PERCENTAGE,12-09-2024 PERCENTAGE,13-09-2024 PERCENTAGE,14-09-2024 PERCENTAGE,15-09-2024 PERCENTAGE,16-09-2024 PERCENTAGE,17-09-2024 PERCENTAGE,18-09-2024 PERCENTAGE,19-09-2024 PERCENTAGE,20-09-2024 PERCENTAGE,21-09-2024 PERCENTAGE,22-09-2024 PERCENTAGE,23-09-2024 PERCENTAGE,24-09-2024 PERCENTAGE,25-09-2024 PERCENTAGE,26-09-2024 PERCENTAGE,27-09-2024 PERCENTAGE,28-09-2024 PERCENTAGE,29-09-2024 PERCENTAGE,30-09-2024 PERCENTAGE,01-10-2024 PERCENTAGE,02-10-2024 PERCENTAGE,03-10-2024 PERCENTAGE,04-10-2024 PERCENTAGE,05-10-2024 PERCENTAGE,06-10-2024 PERCENTAGE,07-10-2024 PERCENTAGE,08-10-2024 PERCENTAGE,09-10-2024 PERCENTAGE,10-10-2024 PERCENTAGE,11-10-2024 PERCENTAGE,12-10-2024 PERCENTAGE,13-10-2024 PERCENTAGE,14-10-2024 PERCENTAGE,15-10-2024 PERCENTAGE,16-10-2024 PERCENTAGE,17-10-2024 PERCENTAGE,18-10-2024 PERCENTAGE,19-10-2024 PERCENTAGE,20-10-2024 PERCENTAGE,21-10-2024 PERCENTAGE,22-10-2024 PERCENTAGE,23-10-2024 PERCENTAGE,24-10-2024 PERCENTAGE,25-10-2024 PERCENTAGE,26-10-2024 PERCENTAGE,27-10-2024 PERCENTAGE,28-10-2024 PERCENTAGE,29-10-2024 PERCENTAGE,30-10-2024 PERCENTAGE,31-10-2024 PERCENTAGE,01-11-2024 PERCENTAGE,02-11-2024 PERCENTAGE,03-11-2024 PERCENTAGE,04-11-2024 PERCENTAGE,05-11-2024 PERCENTAGE,06-11-2024 PERCENTAGE,07-11-2024 PERCENTAGE,08-11-2024 PERCENTAGE,09-11-2024 PERCENTAGE,10-11-2024 PERCENTAGE,11-11-2024 PERCENTAGE,12-11-2024 PERCENTAGE,13-11-2024 PERCENTAGE,14-11-2024 PERCENTAGE,15-11-2024 PERCENTAGE,16-11-2024 PERCENTAGE,17-11-2024 PERCENTAGE,18-11-2024 PERCENTAGE,19-11-2024 PERCENTAGE,20-11-2024 PERCENTAGE,21-11-2024 PERCENTAGE,22-11-2024 PERCENTAGE,23-11-2024 PERCENTAGE,24-11-2024 PERCENTAGE,25-11-2024 PERCENTAGE,26-11-2024 PERCENTAGE
0,Started Troubleshooting (Solution Center),Both,2.449855,2.918847,3.231914,3.057752,2.695824,2.789349,2.723557,3.789585,2.958175,3.078328,3.782263,2.552822,2.532355,2.620244,3.428605,3.127395,2.963751,9.365331,6.472141,2.577243,2.043153,2.901008,2.759946,2.663894,3.062296,2.124639,2.979335,2.512318,2.953576,2.406830,2.311369,3.151260,2.597825,2.311030,4.340565,2.539293,2.942572,3.467884,3.573720,2.782059,2.894648,2.914496,2.451770,2.357750,3.073636,2.681374,3.207651,3.122460,2.367245,2.262106,2.335136,2.829532,2.691077,2.540348,2.826171,3.348387,2.497384,2.630004,2.962642,2.992619,2.949791,2.406186,3.259791,2.930876,3.120187,2.776093,3.745810,2.946854,2.356568,2.522955,2.965013,2.264479,2.494835,2.894942,2.380209,3.100212,2.423803,2.503859,1.872449,2.775544,2.807883,2.012094,2.707533,2.379392,2.724486,2.623764,3.389985,3.897727,3.993493,2.525267
1,Did a Device Reboot,Both,1.049909,1.186490,1.605707,1.326629,1.292321,1.081824,1.212798,1.878732,1.433998,1.347939,1.784141,1.152294,1.344318,1.090075,1.505128,1.574160,1.428928,6.381869,2.637875,0.155536,0.273498,0.423867,0.410069,0.462285,0.363130,0.488653,0.404008,0.295128,0.342513,0.209508,0.307042,0.670554,0.365225,0.315472,0.653654,0.911168,1.051959,1.239886,1.534824,1.013289,1.237943,0.852690,1.044210,0.895319,1.192901,1.258651,1.333482,1.184013,0.751629,0.640600,0.617187,1.036003,0.904368,0.872613,0.878295,1.198846,0.634664,0.824431,1.054636,1.088138,0.808018,0.761510,0.835720,1.209615,1.296575,1.245175,1.365432,0.975912,0.942886,1.078192,1.109287,0.503056,0.864028,0.816780,0.724296,1.118199,0.753806,0.810694,0.623940,0.905954,1.042394,0.493399,0.748051,0.834236,0.671225,0.836754,0.789980,1.603010,1.448567,0.615856
2,Started Proactive and Preventive (P&P),Both,58.565343,56.817169,58.727467,60.319165,59.536396,57.798867,58.021457,57.138063,57.603972,59.758427,62.110545,56.977966,57.782859,59.547130,58.290045,58.249665,59.552043,56.401145,54.047049,59.142272,59.372679,57.821794,58.365708,60.546089,6

In [96]:
# Function to calculate combined metrics (same as in the original code)
def calculate_both_metrics_2(df, new_row, denominator_ios, denominator_android, numerator_ios, numerator_android):
    # Identify the date columns (all columns except 'display_names' and 'operating_system_type')
    date_columns = [col for col in df.columns if col not in ['display_names', 'operating_system_type']]

    # Iterate through each date column to calculate the combined value
    for date in date_columns:
        # Access the date value directly from the rows (this assumes there's only one row for each condition)
        denominator_ios_value = denominator_ios[date].values[0]
        denominator_android_value = denominator_android[date].values[0]
        numerator_ios_value = numerator_ios[date].values[0]
        numerator_android_value = numerator_android[date].values[0]

        # Apply the formula for the combined troubleshooting value
        result = (numerator_ios_value + numerator_android_value) / (denominator_ios_value + denominator_android_value)
        result = result * 100
        # Add the calculated result to the new row
        new_row[date] = result

    # Convert the new row into a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row with the original DataFrame
    df = pd.concat([df, new_row_df], ignore_index=True)

    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', 100)
    
    return new_row_df


In [97]:
# Initialize an empty list to store the result DataFrames
combined_dfs2 = []

# Loop over the config and dynamically fetch numerators, denominators, and new rows
for item in config2:
    numerator_ios = sample_actual_data_df[(sample_actual_data_df['display_names'] == item['numerator_ios']) & (sample_actual_data_df['operating_system_type'] == 'Apple iOS')]
    numerator_android = sample_actual_data_df[(sample_actual_data_df['display_names'] == item['numerator_android']) & (sample_actual_data_df['operating_system_type'] == 'Google Android')]

    denominator_ios = sample_actual_data_df[(sample_actual_data_df['display_names'] == item['denominator_ios']) & (sample_actual_data_df['operating_system_type'] == 'Apple iOS')]
    denominator_android = sample_actual_data_df[(sample_actual_data_df['display_names'] == item['denominator_android']) & (sample_actual_data_df['operating_system_type'] == 'Google Android')]

    # Pass the current configuration to the function
    result_df = calculate_both_metrics_2(
        sample_actual_data_df, 
        item['new_row'], 
        denominator_ios, 
        denominator_android, 
        numerator_ios, 
        numerator_android
    )
    
    # Append the result to the list of DataFrames
    combined_dfs2.append(result_df)

# Concatenate all DataFrames into one
combined_df2 = pd.concat(combined_dfs2, ignore_index=True)

# Show the first 20 rows of the combined DataFrame
combined_df2.head(200)

,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,Saw a Result,Both,96.825397,92.903226,97.452229,95.588235,96.694215,93.877551,96.129032,96.120690,94.478528,90.196078,95.454545,94.354839,95.384615,94.400000,94.512195,95.302013,89.208633,98.790323,96.238245,86.206897,89.690722,86.923077,83.593750,89.256198,91.111111,87.000000,87.741935,85.714286,89.855072,91.304348,82.300885,95.035461,88.888889,85.606061,87.447699,90.076336,84.768212,86.060606,84.615385,83.720930,87.681159,83.673469,91.666667,91.666667,89.552239,96.938776,84.415584,86.274510,90.476190,87.610619,87.142857,90.196078,88.281250,90.076336,88.513514,87.654321,82.786885,94.029851,90.151515,90.909091,84.931507,82.500000,88.461538,88.888889,92.857143,87.500000,85.714286,86.206897,89.523810,90.000000,89.312977,87.301587,88.976378,78.225806,87.826087,87.341772,89.915966,88.489209,93.650794,90.476190,84.090909,83.333333,81.343284,86.324786,86.619718,82.608696,87.500000,87.817259,93.137255,86.178862
1,1st common reason (Unknown),Both,78.688525,80.555556,80.392157,68.461538,76.923077,70.289855,74.496644,76.681614,79.220779,71.014493,74.404762,75.213675,83.870968,76.271186,84.516129,83.098592,77.419355,90.612245,60.912052,11.000000,10.344828,7.964602,6.542056,7.407407,8.130081,9.195402,16.176471,9.803922,11.290323,14.285714,11.827957,8.955224,12.500000,8.849558,8.612440,65.254237,77.343750,69.014085,73.484848,75.000000,66.942149,65.853659,72.727273,70.707071,74.166667,61.052632,66.153846,65.909091,57.894737,58.585859,62.295082,55.072464,61.946903,55.084746,66.412214,58.450704,64.356436,54.761905,57.983193,58.181818,64.516129,49.494949,57.246377,56.250000,71.328671,66.386555,67.816092,64.800000,59.574468,68.686869,54.700855,55.454545,57.522124,47.422680,55.445545,67.391304,57.009346,61.788618,65.254237,57.894737,53.153153,52.941176,49.541284,55.445545,56.910569,57.894737,48.701299,65.317919,61.052632,64.150943
2,2nd common reason (Disconnected),Both,13.114754,11.111111,13.725490,18.461538,17.094017,21.739130,16.778523,13.901345,11.688312,16.666667,19.047619,14.529915,12.903226,14.406780,12.258065,11.971831,12.903226,7.755102,23.127036,1.000000,0.000000,1.769912,0.934579,0.000000,0.000000,1.149425,0.000000,0.980392,0.806452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.474576,10.156250,15.492958,18.181818,18.518519,16.528926,16.260163,15.151515,15.151515,17.500000,22.105263,15.384615,22.727273,26.315789,25.252525,21.311475,34.057971,24.778761,21.186441,25.190840,26.760563,27.722772,30.158730,24.369748,24.545455,27.419355,27.272727,22.463768,23.214286,19.580420,24.369748,21.264368,22.400000,23.404255,19.191919,24.786325,20.000000,25.663717,34.020619,27.722772,18.840580,24.299065,17.886179,18.644068,21.804511,27.927928,29.411765,26.605505,26.732673,21.951220,21.052632,25.974026,13.294798,18.947368,23.584906
3,3rd common reason (Incomplete Install),Both,6.557377,5.555556,6.535948,7.692308,2.564103,5.072464,4.026846,4.932735,3.896104,9.420290,3.571429,5.982906,3.225806,6.779661,3.225806,1.408451,7.258065,1.2244

In [98]:
import pandas as pd

def calculate_sums1(df, items_dict):
    # Sum date columns for each combination of display_names and operating_system_type
    sum_df = df.groupby(['display_names', 'operating_system_type']).sum().reset_index()
    
    # Identify iOS and Android rows
    ios_rows = sum_df[sum_df['operating_system_type'] == 'Apple iOS']
    android_rows = sum_df[sum_df['operating_system_type'] == 'Google Android']
    
    # Initialize an empty list to hold 'Both' rows
    both_rows = []

    # Iterate over each item in the dictionary
    for item in items_dict:
        if any(ios_rows['display_names'].str.contains(item)) or any(android_rows['display_names'].str.contains(item)):
            # Get the relevant iOS and Android rows for this item
            ios_item_row = ios_rows[ios_rows['display_names'].str.contains(item)]
            android_item_row = android_rows[android_rows['display_names'].str.contains(item)]
            
            # Create a new row for 'Both' by summing iOS and Android values
            new_row = {'display_names': item, 'operating_system_type': 'Both'}
            for date in df.columns[2:]:  # Assuming date columns start from index 2
                new_row[date] = ios_item_row[date].sum() + android_item_row[date].sum()
            
            both_rows.append(new_row)

    # Convert both_rows to a DataFrame
    both_df = pd.DataFrame(both_rows)

    # Append the 'Both' rows to the sum_df
    sum_df = pd.concat([sum_df, both_df], ignore_index=True)

    # Drop unnecessary columns if they exist
    sum_df = sum_df.drop(columns=['operating_system_type_ios', 'operating_system_type_Android'], errors='ignore')

    return both_df


In [99]:
both_metrics_df = calculate_sums1(sample_actual_data_df,config3)
both_metrics_df.head(5000)

,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,Visits,Both,529906.0,556078.0,449000.0,424660.0,434478.0,526876.0,569669.0,606958.0,585052.0,448182.0,422761.0,486538.0,502978.0,532932.0,618714.0,579340.0,443351.0,430412.0,481993.0,485386.0,492287.0,516266.0,528907.0,423068.0,405518.0,467249.0,504636.0,513544.0,514886.0,583619.0,470664.0,417791.0,539145.0,580242.0,554112.0,553311.0,541710.0,438094.0,398035.0,461788.0,483156.0,510820.0,522434.0,524207.0,410265.0,401871.0,472226.0,510983.0,518926.0,508299.0,547384.0,420891.0,398126.0,472356.0,492385.0,482373.0,490541.0,514027.0,394457.0,362415.0,443995.0,452838.0,479687.0,485952.0,589227.0,457839.0,480913.0,497549.0,481968.0,474558.0,516173.0,546771.0,429957.0,378069.0,449453.0,483958.0,493497.0,562688.0,666564.0,439120.0,398191.0,479593.0,476018.0,504583.0,499532.0,529193.0,438053.0,401029.0,478504.0,471144.0
1,Visitors,Both,153252.0,160882.0,130135.0,123849.0,127687.0,150687.0,158567.0,169921.0,165994.0,129181.0,122558.0,140200.0,143224.0,152340.0,167112.0,164826.0,128185.0,124927.0,138208.0,135470.0,141339.0,147165.0,152481.0,123485.0,119295.0,134551.0,143509.0,148676.0,145627.0,162076.0,133259.0,121575.0,154716.0,166692.0,154774.0,158912.0,156729.0,128918.0,117881.0,133188.0,137031.0,146238.0,149726.0,152263.0,120248.0,114570.0,135861.0,144788.0,148407.0,146813.0,158286.0,122956.0,117805.0,135984.0,140452.0,138473.0,141865.0,148622.0,117883.0,107466.0,128385.0,130647.0,138321.0,141474.0,168704.0,133251.0,135790.0,140499.0,139198.0,137162.0,147971.0,157127.0,123811.0,115713.0,129008.0,136296.0,141933.0,160023.0,190031.0,127623.0,116911.0,137473.0,136800.0,148467.0,146668.0,155940.0,127759.0,117124.0,140798.0,138296.0
2,Page Views,Both,2503874.0,2661083.0,2157963.0,2003127.0,2064081.0,2499688.0,2725569.0,2955436.0,2830297.0,2156663.0,1973621.0,2248514.0,2315650.0,2460753.0,2783260.0,2675776.0,2098164.0,2046113.0,2253185.0,2184221.0,2312143.0,2323379.0,2477691.0,2001863.0,1878397.0,2163690.0,2346739.0,2386193.0,2376865.0,2665970.0,2180173.0,1900682.0,2465678.0,2710072.0,2527738.0,2598215.0,2543694.0,2062778.0,1840621.0,2121540.0,2181608.0,2352871.0,2400614.0,2446961.0,1923046.0,1663626.0,2183144.0,2291819.0,2409053.0,2363440.0,2558894.0,2008110.0,1845474.0,2215981.0,2314168.0,2239428.0,2288840.0,2396470.0,1877972.0,1677622.0,2058225.0,2100770.0,2237390.0,2253767.0,2772237.0,2204316.0,2239245.0,2317717.0,2228996.0,2206653.0,2399053.0,2553520.0,2087098.0,1776272.0,2114343.0,2217561.0,2325417.0,2669081.0,3058961.0,2166316.0,1891608.0,2248093.0,2268783.0,2447560.0,2416813.0,2568318.0,2191202.0,1930620.0,2267800.0,2286689.0
3,Login Page Visits,Both,168718.0,177196.0,142425.0,135158.0,138430.0,167102.0,181093.0,192538.0,186100.0,141970.0,134341.0,154495.0,159876.0,170052.0,197584.0,185213.0,140685.0,136231.0,152952.0,154420.0,156707.0,164967.0,169065.0,134327.0,128978.0,148397.0,160374.0,163518.0,164231.0,186359.0,149237.0,132711.0,171411.0,184863.0,175945.0,176476.0,172920.0,139168.0,126558.0,146585.0,153492.0,162809.0,16

In [102]:
both_metrics_alldf = calculate_sums1(all_feature_df,config3)
both_metrics_alldf.head(5000)

,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,Visits,Both,995874.0,1045469.0,835949.0,793935.0,815243.0,987819.0,1078252.0,1149854.0,1105978.0,836679.0,787814.0,909168.0,928659.0,1013866.0,1183868.0,1097972.0,825487.0,801352.0,901447.0,885174.0,925829.0,949984.0,994603.0,786126.0,752457.0,870971.0,946632.0,964424.0,964638.0,1107894.0,884985.0,779282.0,1007185.0,1087710.0,1026084.0,1039847.0,1016863.0,812159.0,737692.0,858191.0,882478.0,951651.0,984358.0,985884.0,759614.0,702059.0,882588.0,929465.0,971569.0,952264.0,1022632.0,777570.0,736382.0,878387.0,921519.0,900535.0,917613.0,957619.0,729473.0,671154.0,825918.0,841818.0,896533.0,907010.0,1104240.0,852021.0,897832.0,930012.0,897830.0,886282.0,961175.0,1023597.0,797926.0,699783.0,837332.0,883577.0,923551.0,1062424.0,1194983.0,815580.0,737955.0,889231.0,886765.0,951976.0,932247.0,996081.0,819042.0,747849.0,893439.0,882994.0
1,Visitors,Both,153252.0,160882.0,130135.0,123849.0,127687.0,150687.0,158567.0,169921.0,165994.0,129181.0,122558.0,140200.0,143224.0,152340.0,167112.0,164826.0,128185.0,124927.0,138208.0,135470.0,141339.0,147165.0,152481.0,123485.0,119295.0,134551.0,143509.0,148676.0,145627.0,162076.0,133259.0,121575.0,154716.0,166692.0,154774.0,158912.0,156729.0,128918.0,117881.0,133188.0,137031.0,146238.0,149726.0,152263.0,120248.0,114570.0,135861.0,144788.0,148407.0,146813.0,158286.0,122956.0,117805.0,135984.0,140452.0,138473.0,141865.0,148622.0,117883.0,107466.0,128385.0,130647.0,138321.0,141474.0,168704.0,133251.0,135790.0,140499.0,139198.0,137162.0,147971.0,157127.0,123811.0,115713.0,129008.0,136296.0,141933.0,160023.0,190031.0,127623.0,116911.0,137473.0,136800.0,148467.0,146668.0,155940.0,127759.0,117124.0,140798.0,138296.0
2,Page Views,Both,2503874.0,2661083.0,2157963.0,2003127.0,2064081.0,2499688.0,2725569.0,2955436.0,2830297.0,2156663.0,1973621.0,2248514.0,2315650.0,2460753.0,2783260.0,2675776.0,2098164.0,2046113.0,2253185.0,2184221.0,2312143.0,2323379.0,2477691.0,2001863.0,1878397.0,2163690.0,2346739.0,2386193.0,2376865.0,2665970.0,2180173.0,1900682.0,2465678.0,2710072.0,2527738.0,2598215.0,2543694.0,2062778.0,1840621.0,2121540.0,2181608.0,2352871.0,2400614.0,2446961.0,1923046.0,1663626.0,2183144.0,2291819.0,2409053.0,2363440.0,2558894.0,2008110.0,1845474.0,2215981.0,2314168.0,2239428.0,2288840.0,2396470.0,1877972.0,1677622.0,2058225.0,2100770.0,2237390.0,2253767.0,2772237.0,2204316.0,2239245.0,2317717.0,2228996.0,2206653.0,2399053.0,2553520.0,2087098.0,1776272.0,2114343.0,2217561.0,2325417.0,2669081.0,3058961.0,2166316.0,1891608.0,2248093.0,2268783.0,2447560.0,2416813.0,2568318.0,2191202.0,1930620.0,2267800.0,2286689.0
3,Login Page Visits,Both,168718.0,177196.0,142425.0,135158.0,138430.0,167102.0,181093.0,192538.0,186100.0,141970.0,134341.0,154495.0,159876.0,170052.0,197584.0,185213.0,140685.0,136231.0,152952.0,154420.0,156707.0,164967.0,169065.0,134327.0,128978.0,148397.0,160374.0,163518.0,164231.0,186359.0,149237.0,132711.0,171411.0,184863.0,175945.0,176476.0,172920.0,139168.0,126558.0,146585.0,15